# High-Level API

These are the main objects used in the high-level API to create models, intelligently feed samples into them, and experiment with hyperparameters.

| High-level object |                    Groups together the following objects                   |
|:-----------------:|:--------------------------------------------------------------------------:|
|    `Algorithm`    | Functions to build, train, predict, and evaluate a machine learning model. |
|   `DataPipeline`  |      Dataset, Label, Featureset, Splitset, Foldset, Folds, Preprocess.     |
|    `Experiment`   |    Algorithm, Hyperparamset, Hyperparamcombos, DataPipeline, Batch, Job.   |

In this way, the high-level API abstracts the lower-level API to make things easier for the user.

## Prerequisites
If you've already completed the instructions on the **Installation** page, then let's get started.

In [1]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Usage

### 1. Algorithm

An `Algorithm` is the ORM's codename for a machine learning model since *Model* is the most important *reserved word* for ORMs.

In [2]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense, Dropout

You can name the functions below whatever you want, but do not change the predetermined `*args` (e.g. `**hyperparameters`, `model`, etc.).

Put a placeholder anywhere you want to try out different hyperparameters: `hyperparameters['<some_variable_name>']`. You'll get a chance to define the hyperparameters in a minute.

In [3]:
def function_model_build(**hyperparameters):
    model = Sequential()
    model.add(Dense(hyperparameters['neuron_count'], input_shape=(4,), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(hyperparameters['dropout_size']))
    model.add(Dense(hyperparameters['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(3, activation='softmax', name='output'))

    model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [4]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = 3
        , epochs = hyperparameters['epoch_count']
        , callbacks=[History()]
    )
    return model

Then pass these functions into the `Algorithm`.

The `library` and `analysis_type` help handle the model and its output behind the scenes. Current analysis types include: 'classification_multi', 'classification_binary', and 'regression'.

In [5]:
algorithm = aiqc.Algorithm.make(
	library = "keras"
	, analysis_type = "classification_multi"
	, function_model_build = function_model_build
	, function_model_train = function_model_train
)

#### Hyperparameters

The `hyperparameters` below will be automatically fed into the functions above as `**kwargs` via the `**hyperparameters` argument we saw earlier.

For example, wherever you see `hyperparameters['neuron_count']`, it will pull from the *key:value* pair `"neuron_count": [9, 12]` seen below. Where model A will have 9 neurons and model B will have 12 neurons.

In [6]:
hyperparameters = {
	"neuron_count": [9, 12]
	, "dropout_size": [0.10, 0.20]
    , "epoch_count": [25, 50]
}

### 2. DataPipeline

In [7]:
from sklearn.preprocessing import *

In [8]:
file_path = examples.demo_file_to_pandas('iris.tsv')

In [9]:
datapipeline = aiqc.DataPipeline.make(
	dataFrame_or_filePath = file_path
	, label_column = 'species'
	, size_test = 0.24
	, size_validation = 0.16
	, fold_count = None
	, encoder_features = StandardScaler()
	, encoder_labels = OneHotEncoder(sparse=False)
)

> Don't use `fold_count` unless your (total sample count / fold_count) still gives you an accurate representation of your sample population. You can try it with the 'iris_10x.tsv' demo_file.
>
> In the future, we'll turn the `encoder_*` attributes into customizable functions.

### 3. Experiment

Now it's time to bring together the data and logic into an `Experiment`.

In [10]:
experiment = algorithm.make_experiment(
	datapipeline_id = datapipeline.id
	, hyperparameters = hyperparameters
)

In [11]:
batch = experiment.batch

In [12]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [00:18<00:00,  2.25s/it]


In [15]:
batch.metrics_to_pandas()

job_id       split   roc_auc  accuracy  precision    recall        f1  \
0       41        test  0.991898  0.888889   0.907692  0.888889  0.882963   
1       41  validation  1.000000  0.840000   0.889231  0.840000  0.827879   
2       41       train  0.975395  0.831461   0.853385  0.831461  0.824202   
3       42        test  0.892361  0.805556   0.806527  0.805556  0.805217   
4       42  validation  0.944559  0.840000   0.840000  0.840000  0.840000   
5       42       train  0.911877  0.808989   0.810783  0.808989  0.808659   
6       43        test  0.921296  0.750000   0.781955  0.750000  0.736533   
7       43  validation  0.971765  0.760000   0.791429  0.760000  0.740406   
8       43       train  0.903771  0.674157   0.673975  0.674157  0.635896   
9       44        test  1.000000  0.916667   0.933333  0.916667  0.915344   
10      44  validation  1.000000  0.960000   0.964000  0.960000  0.959719   
11      44       train  0.993573  0.943820   0.952082  0.943820  0.943496   
12      45        test  0.987269  0.833333   0.876320  0.833333  0.818084   
13      45  validation  1.000000  0.880000   0.898990  0.880000  0.871330   
14      45       train  0.965768  0.752809   0.767505  0.752809  0.734817   
15      46        test  0.995370  0.888889   0.916667  0.888889  0.885714   
16      46  validation  0.997647  0.920000   0.934545  0.920000  0.918286   
17      46       train  0.979940  0.842697   0.866033  0.842697  0.838533   
18      47        test  0.870370  0.583333   0.484848  0.583333  0.514919   
19      47  validation  0.921029  0.760000   0.781538  0.760000  0.741818   
20      47       train  0.923811  0.685393   0.713869  0.685393  0.637581   
21      48        test  0.989583  0.944444   0.952381  0.944444  0.944056   
22      48  validation  0.963824  0.880000   0.883889  0.880000  0.880000   
23      48       train  0.933971  0.786517   0.786530  0.786517  0.786394   

        loss  
0   0.395283  
1   0.390014  
2   0.445891  
3   0.486446  
4   0.431892  
5   0.472854  
6   0.654026  
7   0.559443  
8   0.684904  
9   0.365426  
10  0.360211  
11  0.391472  
12  0.579135  
13  0.517191  
14  0.594523  
15  0.302407  
16  0.304320  
17  0.340107  
18  0.687662  
19  0.611071  
20  0.631250  
21  0.366575  
22  0.422384  
23  0.452266

### Metrics & Visualization

For more information of visualization of performance metrics, reference the **Visualization & Metrics** documentation.